### 任務類型
檢索增強生成 (Retrieval-Augmented Generation, RAG)

### 任務目標
建立一個本地端的 AI Agent。它會讀取collect.ipynb產生的 vault.txt 檔案，使用 Ollama 上的本地嵌入模型 (nomic-embed-text) 將文件內容轉換為向量，並儲存在 Chroma 向量資料庫中。接著，它啟動一個互動式聊天介面，讓使用者提問，腳本會先從向量資料庫中檢索相關的郵件內容片段，再將這些內容連同問題一起交給本地的 LLM (llama3.1:8b) 來生成回答。

### 資料集
collect.ipynb產生的 vault.txt檔案。

### 步驟 1｜匯入所有需要的函式庫
**TextLoader**: 用於載入 vault.txt。

**OllamaEmbeddings**: 用於連接本地 Ollama 的嵌入模型。

**Chroma**: 用於本地向量資料庫。

**ChatOllama**: 用於連接本地 Ollama 的大型語言模型 (LLM)。

**RecursiveCharacterTextSplitter**: 用於將載入的 vault.txt 內容切塊。

**ChatPromptTemplate, RunnableParallel, StrOutputParser** 等：LangChain 運作鏈 (LCEL) 的核心組件，用於構建 RAG 流程。

In [8]:
import os
import shutil
import yaml
import logging
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma 
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

### 步驟 1.5｜美化輸出增加可讀性
定義 ANSI 顏色代碼，用於在終端機中以不同顏色(例如 NEON_GREEN)印出 AI 的回應，增加可讀性。

In [9]:
# ANSI escape codes for colors
PINK = "\033[95m"
CYAN = "\033[96m"
NEON_GREEN = "\033[92m"
RESET_COLOR = "\033[0m"

### 步驟 2｜設定logging的格式
**load_config(config_file)**:讀取config.yaml中的設定參數。

**logging.basicConfig()**:設定logging的基本組態，讓記錄包含時間戳、等級和訊息。

In [ ]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def load_config(config_file):
    logging.info(f"Loading configuration from '{config_file}'...")
    try:
        with open(config_file, "r", encoding="utf-8") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        logging.error(f"Configuration file '{config_file}' not found.")
        exit(1)
    except Exception as e:
        logging.error(f"Error loading configuration: {e}")
        exit(1)


### 步驟 3｜設計內容向量的產生流程
**lload_or_create_vector_store()**:

1. 載入文件: 使用 TextLoader 載入 vault.txt 並用 RecursiveCharacterTextSplitter 切塊。

2. 檢查快取: 檢查 vector_store_path（./chroma_db_notebook）是否存在。

3. 建立新 DB: 如果不存在，它會呼叫 Chroma.from_documents，這會花費一些時間來為所有文件塊生成嵌入向量，並將它們儲存到磁碟。

4. 載入舊 DB (並更新): 如果已存在，它會直接載入。接著，它會執行一個「增量更新」檢查：它比較 vault.txt 中的「當前」文件塊和資料庫中「已存在」的文件塊。如果發現 vault.txt 中有新的內容，它只會將「新」的文件塊（new_docs_to_add）加入到資料庫中，而不是每次都重建整個資料庫。

In [ ]:
def load_or_create_vector_store(config, embeddings_model, clear_cache=False):
    vector_store_path = config["vector_store_path"]

    if clear_cache and os.path.exists(vector_store_path):
        logging.info(f"Clearing cache at '{vector_store_path}'...")
        shutil.rmtree(vector_store_path)

    vault_file = config["vault_file"]
    if not os.path.exists(vault_file):
        logging.error(f"Vault file '{vault_file}' not found.")
        if not os.path.exists(vector_store_path):
            logging.error("No vault file and no existing vector store. Exiting.")
            exit(1)

        logging.warning(
            "Vault file not found, loading existing vector store without updates."
        )
        return Chroma(
            persist_directory=vector_store_path, embedding_function=embeddings_model
        )

    logging.info(f"Loading documents from '{vault_file}'...")
    loader = TextLoader(vault_file, encoding="utf-8")
    documents = loader.load()

    logging.info("Splitting documents recursively...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", " ", ""]
    )
    current_docs = text_splitter.split_documents(documents)

    if not os.path.exists(vector_store_path):
        logging.info("Creating new vector store...")

        if not current_docs:
            logging.error(
                "No document chunks found after splitting. Is 'vault.txt' empty?"
            )
            exit(1)

        logging.info(
            f"Found {len(current_docs)} document chunks. Generating embeddings..."
        )

        try:
            db = Chroma.from_documents(
                current_docs, embeddings_model, persist_directory=vector_store_path
            )
            logging.info(f"Vector store created and saved to '{vector_store_path}'.")
            return db
        except Exception as e:
            logging.error(
                f"Failed to create embeddings or vector store: {e}", exc_info=True
            )
            exit(1)
    else:
        logging.info(f"Loading existing vector store from '{vector_store_path}'...")
        try:
            db = Chroma(
                persist_directory=vector_store_path, embedding_function=embeddings_model
            )

            logging.info("Checking for document updates...")
            existing_data = db.get(include=["documents"])
            existing_docs_content = set(existing_data["documents"])

            if not existing_docs_content:
                logging.warning("Existing vector store is empty. Re-populating...")
                db.add_documents(current_docs)
                logging.info("Vector store re-populated.")
                return db
            new_docs_to_add = []
            for doc in current_docs:
                if doc.page_content not in existing_docs_content:
                    new_docs_to_add.append(doc)
            if new_docs_to_add:
                logging.info(
                    f"Found {len(new_docs_to_add)} new document chunks. Adding to vector store..."
                )
                db.add_documents(new_docs_to_add)
                logging.info("Vector store successfully updated.")
            else:
                logging.info("Vector store is already up-to-date.")

            return db

        except Exception as e:
            logging.error(
                f"Failed to load or update vector store from {vector_store_path}: {e}",
                exc_info=True,
            )
            exit(1)

### 步驟 4｜定義主程式
**format_docs()**:
將檢索到的文件列表轉為單一字串。
**main()**:
1. 初始化: 載入設定檔、初始化 OllamaEmbeddings 和 ChatOllama 模型。

2. 載入/更新 DB: 呼叫 load_or_create_vector_store 來準備好向量資料庫和檢索器 (retriever)。

3. 定義提示詞: 建立 ChatPromptTemplate，它包含了系統訊息、聊天歷史 (history)，以及一個組合了「上下文 (context)」和「問題 (question)」的區塊。

4. 建立 RAG 鏈: 這是最關鍵的部分。使用 RunnableParallel 來定義 RAG 流程：

    - 當使用者輸入 question 時，系統「同時」執行三件事：

        - context: 將 question 傳給 retriever 進行檢索，並用 format_docs 格式化。

        - question: 將 question 原樣傳遞下去。

        - history: 將 history 變數原樣傳遞下去。

    - 將這三樣東西 (context, question, history) 一起餵給 prompt。

    - 將格式化後的提示詞交給 llm 生成答案。

    - 最後用 StrOutputParser 取得純文字回應。

5. 啟動聊天: 進入 while True 迴圈，等待使用者輸入。

6. 維護歷史: 每次問答後，將使用者的 HumanMessage 和 AI 的 AIMessage 存入 chat_history 列表，以便在下次提問時提供上下文記憶。


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def main():

    CONFIG_FILE = "configwj.yaml"
    CLEAR_CACHE = False

    config = load_config(CONFIG_FILE)
    

    embeddings_model = OllamaEmbeddings(model=config["embed_model"])

    llm = ChatOllama(model=config["ollama_model"])

    db = load_or_create_vector_store(config, embeddings_model, CLEAR_CACHE)

    retriever = db.as_retriever(search_kwargs={"k": config["top_k"]})

    chat_history = []

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", config["system_message"]),
            MessagesPlaceholder(variable_name="history"),
            (
                "human",
                "Based on the following context from my documents:\n"
                "--- CONTEXT ---\n"
                "{context}\n"
                "--- END CONTEXT ---\n\n"
                "My question is: {question}",
            ),
        ]
    )

    rag_chain = (
        RunnableParallel(
            {
                "context": (lambda x: x["question"]) | retriever | format_docs,
                "question": (lambda x: x["question"]),
                "history": (lambda x: x["history"]),
            }
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    logging.info(
        f"Chatbot initialized with model '{config['ollama_model']}'. Type 'quit' to exit."
    )


    while True:
        try:
            user_input = input(
                "Ask a question about your documents: "
            )
            if user_input.lower() == "quit":
                break

            if not user_input.strip():
                continue

            inputs = {"question": user_input, "history": chat_history}
            response = rag_chain.invoke(inputs)
            print(NEON_GREEN + "Response: \n\n" + response + RESET_COLOR)

            chat_history.append(HumanMessage(content=user_input))
            chat_history.append(AIMessage(content=response))

        except KeyboardInterrupt:
            print("\nExiting...")
            break
        except Exception as e:
            logging.error(f"An error occurred during chat: {e}", exc_info=True)

In [13]:
main()

2025-10-28 23:30:56,148 - INFO - Loading configuration from 'configwj.yaml'...
2025-10-28 23:30:56,168 - INFO - Loading documents from 'vault.txt'...
2025-10-28 23:30:56,195 - INFO - Splitting documents recursively...
2025-10-28 23:30:56,201 - INFO - Loading existing vector store from './chroma_db_notebook'...
2025-10-28 23:30:56,347 - INFO - Checking for document updates...
2025-10-28 23:30:56,354 - INFO - Vector store is already up-to-date.
2025-10-28 23:30:56,355 - INFO - Chatbot initialized with model 'llama3.1:8b'. Type 'quit' to exit.
